In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import arrayfire as af
import numpy as np

from dg_maxwell import params
from dg_maxwell import utils

af.set_backend(params.backend)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [2]:
def polynomial_product_coeffs(poly1_coeffs, poly2_coeffs):
    '''
    '''
    poly1_coeffs_tile = af.transpose(af.tile(poly1_coeffs, 1, poly1_coeffs.shape[0]))
    poly2_coeffs_tile = af.tile(poly2_coeffs, 1, poly2_coeffs.shape[0])

    product_coeffs = poly1_coeffs_tile * poly2_coeffs_tile

    return product_coeffs

In [3]:
foo_degree = 4
foo = af.flip(af.np_to_af_array(np.arange(1, foo_degree)))

bar_degree = 4
bar = af.flip(af.np_to_af_array(np.arange(1, bar_degree)))

In [4]:
x = utils.linspace(-1, 1, 8)
y = utils.linspace(-1, 1, 8)

In [5]:
poly_xy = polynomial_product_coeffs(foo, bar)
poly_xy = af.tile(poly_xy, d0 = 1, d1 = 1, d2 = x.shape[0])
print(poly_xy)

arrayfire.Array()
Type: long int

[3 3 8 1]
         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 

         9          6          3 
         6          4          2 
         3          2          1 





In [6]:
x_power = af.flip(af.range(poly_xy.shape[1], dtype = af.Dtype.u32))
x_power = af.tile(af.transpose(x_power), d0 = poly_xy.shape[0])
x_power = af.tile(x_power, d0 = 1, d1 = 1, d2 = x.shape[0])
# print(x_power)

In [7]:
y_power = af.flip(af.range(poly_xy.shape[0], dtype = af.Dtype.u32))
y_power = af.tile(y_power, d0 = 1, d1 = poly_xy.shape[1])
y_power = af.tile(y_power, d0 = 1, d1 = 1, d2 = y.shape[0])
# print(y_power)

In [8]:
X = af.reorder(x, d0 = 2, d1 = 1, d2 = 0)
X = af.tile(X, d0 = poly_xy.shape[0], d1 = poly_xy.shape[1])
X = X**x_power
# print(X**x_power)

Y = af.reorder(y, d0 = 2, d1 = 1, d2 = 0)
Y = af.tile(Y, d0 = poly_xy.shape[0], d1 = poly_xy.shape[1])
Y = Y**y_power
# print(Y**y_power)

In [9]:
af.display(x, 14)

In [10]:
XY = X * Y
poly_val = poly_xy * XY
poly_val = af.sum(af.sum(poly_val, dim = 0), dim = 1)
poly_val = af.reorder(poly_val, d0 = 2, d1 = 1, d2 = 0)
# print(Y[:, :, 0])
# print(poly_xy[:, :, 0])
af.display(poly_val, 14)

In [11]:
print(utils.polyval_2d(polynomial_product_coeffs(foo, bar), x, y))

arrayfire.Array()
Type: double

[1 8 1 1]
    4.0000     1.2145     0.4815     0.6014     1.8142     5.7993    15.6751    36.0000 



In [12]:
multiple_poly = af.tile(polynomial_product_coeffs(foo, bar), d0 = 1, d1 = 1, d2 = 3)
poly_2d = multiple_poly
xi = x
eta = y

In [13]:
utils.polyval_2d(multiple_poly, x, y)

arrayfire.Array()
Type: double
Dims: (3, 8)